<a href="https://colab.research.google.com/github/yaniv92648/ITC_Final_Project_Personality_Types/blob/master/ITC_Final_Project_Personality_Types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('MBTI.csv.zip', compression='zip')

In [8]:
data

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
...,...,...
8670,ISFP,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,ENFP,'So...if this thread already exists someplace ...
8672,INTP,'So many questions when i do these things. I ...
8673,INFP,'I am very conflicted right now when it comes ...


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


In [10]:
data.describe()

,type,posts
count,8675,8675
unique,16,8675
top,INFP,'There are other things in the same vain as so...
freq,1832,1


In [0]:
TWITTER_API_KEY = 'tj9zC2ySqcrmKlbscFg97aGzj'
TWITTER_API_SECRET_KEY = 'JjlTbrxGWghmUuQttD7Eq0rbZ4iDkqwKazOT5epdgvPUds0zNC'